<a href="https://colab.research.google.com/github/8persy/algoritms_colab/blob/main/%22AA_%D0%B4%D0%B5%D1%80%D0%B5%D0%B2%D0%BE_11_311_11_313_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# АА-дерево

Модификация красно-черного дерева с целью упрощения реализации

Будем использовать ту же ноду, что и для КЧД

In [ ]:
from __future__ import annotations
from dataclasses import dataclass, field
from typing import Any, Literal


@dataclass
class Node:
    key: Any
    parent: Node | None = field(default=None)
    right: Node | None = field(default=None)
    left: Node | None = field(default=None)
    height: int = field(default=1)
    color: Literal["red", "black"] = field(default="black")

Правила АА-дерева:
1. Цвет вершины может быть черный или красный
2. Корень всегда черный
3. Листья всегда черные
4. Каждая красная вершина должна иметь черных сыновей
5. Пути от вершины к ее листьям должны содержать одинаковое количество черных вершин (черная высота)
6. В дереве не может быть левого красного сына

Также в АА-дереве меняется понятие высоты

Высота здесь это не количество нод от корня до узла, а отдельная величина для узла и увеличивается посредством операций при перебалансировке

Как определяется высота:
- У None листьев высота = 0
- У листьев высота = 1
- У красных нод высота та же, что и у его родителя
- У черных нод высота на 1 меньше, чем у их родителей

<img src="https://drive.google.com/uc?export=view&id=1WNEMPmps800xH20k4yVhbZT4JpC70Q_D"/>

Высота 1: 8, 9, 24, 35, ...\
Высота 2: 18, 29, 57, 72\
Высота 3: 46, 68

Связи м-ду нодами на одной высоте (46 и 68 например) называются **горизонтальными**

Горизонтальная связь всегда ведет к красной ноде

## Вставка элемента

Делается по аналогии с красно-черным деревом
- Ищем куда добавить ноду
- Добавляем красную ноду
- Производим балансировку

### Балансировка

Благодаря новому условию, проблемы вызывают всего 3 (вместо прошлых 7) кейса, давайте их рассмотрим

Попробуем составить дерево по следующему массиву:\
``[57, 55, 72, 18, 46, 68, 24, 9, 59, 97, 29, 35, 89,  8]``

#### Случай 1. Красный корень

По аналогии с КЧД, красим ноду в черный

#### Случай 2. Левая горизонтальная связь

##### Если родитель черный

<img src="https://drive.google.com/uc?export=view&id=1Qly-u_LTBGA-a3t2SwRpHSjdAhnphNEe" height="200"/>

Обобщим

<img src="https://drive.google.com/uc?export=view&id=1DmVScSj0gPx99r7gOrLOKHTxH0Bwqpdh" height="400"/>



Что делать в таком случае?

- Правый поворот м-ду левым сыном (A) и отцом (B)
- Перекрашиваем сына в черное
- Перекрашиваем отца в красное

**Высота в этом случае ни у кого не изменяется**

Назовем это преобразование **skew**

Благодаря тому, что последнее правило АА-дерево несимметрично (левого красного сына быть не может), симметричный кейс проблем у нас не вызывает

<img src="https://drive.google.com/uc?export=view&id=16k5L4T6uVaw8dgqcOgbZp9Qd827oRYkJ" height="400"/>

<img src="https://drive.google.com/uc?export=view&id=1lH1IxiVf9pyEaTN6VrO9lAhgnWO_w8lm" height="200"/>

##### Чекпоинт 1. skew

In [ ]:
def skew(node: Node) -> Node:
  child = node.left

  subtree = child.right

  node.left = subtree
  subtree.parent = node

  child.right = node
  child.parent = node.prent
  node.parent = child

  child.color = 'black'
  node.color = 'red'

  return child

##### Если родитель красный

Исходное дерево, добавляем **46**

<img src="https://drive.google.com/uc?export=view&id=1yNUB-P-iIGyg5ekiJjZjhlZT7rKWOmPQ" height="400"/>

<img src="https://drive.google.com/uc?export=view&id=1oBjM3F2t-roeZPIW6PpIbvLC4tKeTyzO" height="400"/>

Ноды **55** и **46** - две красные ноды на одной высоте

Что делать?
- Правый поворот м-ду сыном и отцом

Перекраска в этом случае не делается

<img src="https://drive.google.com/uc?export=view&id=1yPhM5-EEuvSTff8E4iNIgW5PkLxpGECH" height="400"/>

В этой получим новый случай проблемы балансировки, разберем его ниже

##### Чекпоинт 1.1 Доработать skew

In [ ]:
def skew(node: Node) -> Node:
  child = node.left

  subtree = child.right

  node.left = subtree
  subtree.parent = node

  child.right = node
  child.parent = node.prent
  node.parent = child

  if  node.color != 'red':
    child.color = 'black'
    node.color = 'red'

  return child

#### Случай 3. Двойная горизонтальная связь

Речь пойдет о двойной **правой** горизонтальной связи\
Двойная левая горизонтальная связь невозможна, фиксится как 2 случая левой горизонтальной связи


<img src="https://drive.google.com/uc?export=view&id=1RbNv3BWXVU3Ur561IUSEFbVWeVB8uq4T" height="200"/>

Обобщим

<img src="https://drive.google.com/uc?export=view&id=1opiMtrq3r2qYx0lzkUVBprvu15ijEU2g" height="400"/>

Что делаем?
- Левый поворот м-ду A и B
- Увеличиваем высоту у B
- Перекрашиваем C в черный

В этом случае высота увеличивается **только у B**

Назовем это преобразование **split**

<img src="https://drive.google.com/uc?export=view&id=1agOvmWfkLViJDVN8V8NIudLDwIPqFEdT" height="600"/>

По итогу преобразования можем получить 3 случая:
- B становится корнем дерева
  - Обрабатываем как первый случай
- B образовывает **левую** горизонтальную связь
  - Обрабатываем как второй случай
- B образовывает **правую** горизонтальную связь
  - Тут проблем нет

<img src="https://drive.google.com/uc?export=view&id=1cxgaji_EYRvBhoSf6s07hrrKl9HiUZmg" height="400"/>

##### Чекпоинт 2. split

In [ ]:
def split(node: Node) -> Node:
  child = node.right

  subtree = child.left

  child.parent = node.parent
  node.parent = child

  child.left = node
  node.right = subtree

  child.height += 1

  child.right.color = 'black'

  return child

### Соберем все в кучу

In [ ]:
def insert_fix(tree: Node, node: Node):
  while node.parent is not None:
    if node.left is not None and node.left.color == "red":
      node = skew(node)
    elif node.right is not None and node.right.color == "red" and node.right.right is not None and node.right.right.color == "red":
      node = split(node)
    else:
      node = node.parent
    tree.color = "black"

In [ ]:
def insert(tree: Node, key: Any):
  parent = None
  curr = tree

  while curr is not None:
    parent = curr
    if key < curr.key:
      curr = curr.left
    else:
      curr = curr.right

  node = Node(key=key, color="red")

  if parent is None:
    tree = node
  else:
    node.parent = parent
    node.height = parent.height
    if key < parent.key:
      parent.left = node
    else:
      parent.right = node

  insert_fix(tree, node)

## Удаление элемента

Рассмотрим 4 возможных случая

Будем рассматривать случаи на следующем дереве

<img src="https://drive.google.com/uc?export=view&id=1kQj9u34pNae9Kp9qX9ypCxHhFD949yIp" height="400"/>

### Случай 1. Удаляем красный лист

Просто удаляем лист, ничего больше не нужно

В нашем примере - нода **9**

<img src="https://drive.google.com/uc?export=view&id=1gjwB0_RsynZukxvg6LlrBP9lsY-BgyN1" height="400"/>

### Случай 2. Удаляем черный лист с красной нодой

- Удаляем ребенка
- Заменяем значение ноды на значение ребенка
- Перекрашиваем ноду в черный

В нашем примере - нода **8**

<img src="https://drive.google.com/uc?export=view&id=1kQj9u34pNae9Kp9qX9ypCxHhFD949yIp" height="400"/>

<img src="https://drive.google.com/uc?export=view&id=1IF8fvz_IxaqnDfnK93MOD9y7fri_XG-s" height="400"/>

### Чекпоинт 3

In [ ]:
# Функция удаления красного листа
# node - красный лист
def delete_red_leaf(node: Node):
  parent = node.parent
  parent.right = None

  # не обязательно :)
  node.parent = None


In [ ]:
# Функция удаления черного листа с красной нодой
# node - черный лист
def remove_black_leaf_red_right(node: Node):
  node.key = node.right.key
  delete_red_leaf(node.right)

### Случай 3. Удаляем ноду с двумя детьми

Для замены значения в удаляемой ноде нам необходимо найти некоторый узел\
Назовем этот узел **наследником**

В зависимости от реализации наследниками могут быть:
- Минимальное значение, большее чем текущее ("потомок")
- Максимальное значение, меньшее чем текущее ("предшественник")

Для реализации будем трактовать наследника как потомка

Заменяем удаляемую ноду наследником и обрабатываем наследника:
- Если наследник - черный лист с красной нодой, обрабатываем как второй случай
- Иначе - см. случай 4

In [ ]:
def find_successor(node: Node):
  successor = node.right

  while successor.left is not None:
    successor = successor.left

  return successor

##### Наследник с красной нодой

Удалим ноду **18**\
Наследник для нее - нода **24**

<img src="https://drive.google.com/uc?export=view&id=1kQj9u34pNae9Kp9qX9ypCxHhFD949yIp" height="400"/>

<img src="https://drive.google.com/uc?export=view&id=1BFOQIaxcOz7oykyItJ8xxDC26Wg7-5CQ" height="400"/>

#### Чекпоинт 4

In [ ]:
# Функция удаления ноды, наследник с красной нодой
# node - удаляемая нода
def delete_with_red_node_successor(node: Node)
  successor = find_successor(node)

  node.key = successor.key
  remove_black_leaf_red_right(successor)

### Случай 4. Удаляем черный лист

- Удаляем ноду
- Идем от удаленной ноды до корня
- Для каждой ноды ($p$) на пути с двумя детьми делаем следующее:
  - Если есть ребенок с высотой с разницей в 2 по сравнению с текущим:
    - Уменьшаем высоту $p$
    - Если правый ребенок - красный, уменьшаем и его высоту
  - При необходимости делаем
    - skew($p$)
    - skew($p.right$)
    - skew($p.right.right$)
    - split($p$)
    - split($p.right$)
  - Если $p$ - красный и у родителя высота больше, красим $p$ в черный

При уменьшении высоты, красим детей той же высоты в красный цвет

Для примера возьмем построенное выше дерево и попробуем удалить корень - **46**

<img src="https://drive.google.com/uc?export=view&id=1rKNemlMOx84QT7zNZE1KEcC10gU9b_Ri" height="200"/>

Наследник корня - **55**\
Заменим значение и удалим наследника

<img src="https://drive.google.com/uc?export=view&id=1xSjBOG9bAW7UCn2uG0mhNJD03R4OS7Nj" height="200"/>

У ноды 57 есть левый ребенок, который отличается по высоте от родителя на 2\
(у None высота = 0, у ноды 57 высота = 2)

Уменьшаем высоту у ноды 57\
Тк у 57 есть правый ребенок высоты 1, красим его в красный

<img src="https://drive.google.com/uc?export=view&id=1xmxqC10x0ooC6DSlbzLvoBYEYTxgNgZL" height="200"/>

Заметим что у ноды 68 есть ребенок, отличающийся по высоте на 2 (нода 57)\
Понизим высоту у ноды 68

<img src="https://drive.google.com/uc?export=view&id=1tqEsiPCe9xqNzTj8aq4QsR0MFZqHBm0L" height="200"/>

Покрасим ноды на одной высоте в красный

<img src="https://drive.google.com/uc?export=view&id=1i7z_iE1wJlM8WSUgBhVl7sk9HYrXQ8NS" height="200"/>

Нода 68 красная, у ноды 55 высота на 1 выше => красим 68 в черное

<img src="https://drive.google.com/uc?export=view&id=1xDdU2pKF4KLPQu2-YEWZm9rYrGKuGBA_" height="200"/>

In [ ]:
def get_height(node: Node | None):
  if node is None:
    return 0
  return node.height

def check_skew(node: Node):
  return node.left.height == node.height and node.left.color == "red"

def check_split(node: Node):
  child = node.right
  grandson = child.right

  return node.height == child.height and child.height == grandson.height and child.color == "red" and grandson.color == "red"

def decrease_height(node: Node):
  node.height -= 1
  if get_height(node.right) == node.height:
    node.right = "red"
  elif get_height(node.left) == node.height:
    node.left = "red"
  return node

def delete_black_leaf(node: Node):
  successor = find_successor(node)

  node.key = successor.key

  p = successor.parent

  if p.left == successor:
    p.left = None
  else:
    p.right = None

  while p.parent is not None:
    height = get_height(p)
    if height - get_height(p.right) > 1 or height - get_height(p.left) > 1:
      p = decrease_height(p)
      if get_height(p.right) > node.height or p.right.color == "red":
        decrease_height(node.right)
    if p.left is not None and check_skew(p):
      p = skew(p)
    if p.right is not None and p.right.left is not None and check_skew(p.right):
      p.right = skew(p.right)
    if p.right is not None and p.right.right is not None and p.right.right.left is not None and check_skew(p.right.right):
      p.right.right = skew(p.right.right)
    if p.right is not None and p.right.right is not None and check_split(p):
      p = split(p)
    if p.right is not None and p.right.right is not None and p.right.right.right is not None and check_split(p.right):
      p.right = split(p.right)
    if p.color == "red" and p.parent.height > p.height:
      p.color = "black"

    p = p.parent

### Собираем все в кучу

In [ ]:
def delete(tree: Node, key: Any):
  node = tree

  if key < node.key:
    node = node.left
  elif key > node.key:
    node = node.right
  else:
    if node.color == "red" and node.height == 1:
      delete_red_leaf(node)
    elif node.color == "black" and node.height == 1 and node.right is not None:
      remove_black_leaf_red_right(node)
    elif node.right is not None and node.left is not None:
      successor = find_successor(node)

      if successor.height == 1 and successor.right is not None:
        delete_with_red_node_successor(node)
      else:
        delete_black_leaf(node)
    else:
      delete_black_leaf(node)